In [1]:
import scipy.io
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from pprint import pprint

In [2]:
harold_list = []
jakob_list = []
filepath = r"C:\Users\harol\Desktop\MSc Cognitive Science & Artificial Intelligence\Spring 2022\Brain Computer Interfacing\Project\RPS EEG Recordings"
for file in os.listdir(filepath):
    if file.startswith("Harold"):
        harold_list.append(file)
    if file.startswith("Jakob"):
        jakob_list.append(file)
harold_list.append(harold_list[0])
_ = harold_list.pop(0)

jakob_list.append(jakob_list[0])
_ = jakob_list.pop(0)
jakob_list.append(jakob_list[0])
_ = jakob_list.pop(0)
jakob_list.append(jakob_list[0])
_ = jakob_list.pop(0)

print(harold_list)
print(len(harold_list))

print(jakob_list)
print(len(jakob_list))

['HaroldReal1_13_05_2022_12_16_32_0000.mat', 'HaroldReal2_13_05_2022_12_50_44_0000.mat', 'HaroldReal3_13_05_2022_13_04_46_0000.mat', 'HaroldReal4_13_05_2022_13_20_10_0000.mat', 'HaroldReal5_13_05_2022_13_31_41_0000.mat', 'HaroldReal6_18_05_2022_13_44_19_0000.mat', 'HaroldReal7_18_05_2022_14_00_22_0000.mat', 'HaroldReal8_18_05_2022_14_09_39_0000.mat', 'HaroldReal9_18_05_2022_14_23_33_0000.mat', 'HaroldReal10_18_05_2022_14_33_46_0000.mat']
10
['JakobReal3_18_05_2022_15_13_21_0000.mat', 'JakobReal4_18_05_2022_15_24_44_0000.mat', 'JakobReal5_18_05_2022_15_46_06_0000.mat', 'JakobReal6_18_05_2022_15_59_06_0000.mat', 'JakobReal7_18_05_2022_16_12_18_0000.mat', 'JakobReal8_30_05_2022_11_41_28_0000.mat', 'JakobReal9_30_05_2022_11_53_55_0000.mat', 'JakobReal10_30_05_2022_12_11_01_0000.mat', 'JakobReal11_30_05_2022_12_23_58_0000.mat', 'JakobReal12_30_05_2022_12_36_11_0000.mat']
10


In [3]:
dataframe_list = []
name = "Jakob"
#name = "Harold"
if name == "Jakob":
    print(name)
    for index, file in enumerate(jakob_list):
        old_len = len(dataframe_list)
        mat = scipy.io.loadmat(os.path.join(filepath, file))
        print(mat["y"][31][0])
        df = pd.DataFrame(mat['y']).transpose()
        df.rename(columns={0: "Time", 31: "Marker"}, inplace=True)
        
        df_cleaned = df[df.Marker != mat["y"][31][0]]
        # get marker indices
        marker_list = []
        for row in range(df_cleaned.shape[0]):
            # store when 'Marker' value switches from 0 to 1
            if df_cleaned["Marker"].iloc[row] == 1.0 and df_cleaned["Marker"].iloc[row-1] == 0:
                marker_list.append(row)

        #gives starting index of each trial
        print(f'Marker list (length {len(marker_list)}):')
        #print(marker_list)
        
        # Cut up each trial into its own dataframe, up to 1625 frames per trial to remove the rest state
        for marker_index in range(len(marker_list)):
            #print(f"Markers trial {marker_index}:\t", str(marker_list[marker_index-1]), str(marker_list[marker_index]))
            
            #Get only the first 1625 frames after the start of each trial
            result = df_cleaned.iloc[marker_list[marker_index]:marker_list[marker_index] + 1625]
            
            
            dataframe_list.append(result.T)
        print("Number of trials found: " + str(len(dataframe_list)-old_len))
        
        
if name == "Harold":
    print(name)
    for index, file in enumerate(harold_list):
        old_len = len(dataframe_list)
        mat = scipy.io.loadmat(os.path.join(filepath, file))
        
        #This differs per trial!
        print(mat["y"][31][0])
        df = pd.DataFrame(mat['y']).transpose()
        df.rename(columns={0: "Time", 31: "Marker"}, inplace=True)

        df_cleaned = df[df.Marker != mat["y"][31][0]]
        # get marker indices
        marker_list = []
        for row in range(df_cleaned.shape[0]):
            # store when 'Marker' value switches from 0 to 1
            if df_cleaned["Marker"].iloc[row] == 1.0 and df_cleaned["Marker"].iloc[row-1] == 0:
                marker_list.append(row)

        #gives starting index of each trial
        print(f'Marker list (length {len(marker_list)}):')
        #print(marker_list)

        # Cut up each trial into its own dataframe, up to 1625 frames per trial to remove the rest state
        for marker_index in range(len(marker_list)):
            #print(f"Markers trial {marker_index}:\t", str(marker_list[marker_index-1]), str(marker_list[marker_index]))

            #Get only the first 1625 frames after the start of each trial
            result = df_cleaned.iloc[marker_list[marker_index]
                :marker_list[marker_index] + 1625]

            dataframe_list.append(result.T)
        print("Number of trials found: " + str(len(dataframe_list)-old_len))
        
for index, i in enumerate(dataframe_list):
            if i.shape != (32, 1625):
                print("Issue at " + str(index))

np_array = np.array(list(map(lambda x: x.to_numpy(), dataframe_list)))
print("Final Array Shape: " + str(np_array.shape))
file_name = f"{name}ProcessedData.npy"
out_path = r"C:\Users\\harol\\Desktop\\MSc Cognitive Science & Artificial Intelligence\\Spring 2022\\Brain Computer Interfacing\\Project\\"
np.save(os.path.join(out_path, file_name), np_array)
    

Jakob
-0.3
Marker list (length 45):
Number of trials found: 45
-0.4
Marker list (length 45):
Number of trials found: 45
-0.1
Marker list (length 45):
Number of trials found: 45
-0.2
Marker list (length 45):
Number of trials found: 45
-0.3
Marker list (length 45):
Number of trials found: 45
-0.4
Marker list (length 45):
Number of trials found: 45
-0.1
Marker list (length 45):
Number of trials found: 45
-0.2
Marker list (length 45):
Number of trials found: 45
-0.3
Marker list (length 45):
Number of trials found: 45
-0.4
Marker list (length 45):
Number of trials found: 45
Final Array Shape: (450, 32, 1625)


In [6]:
os.path.join(filepath, harold_list[0])
mat = scipy.io.loadmat(os.path.join(filepath, file))
df = pd.DataFrame(mat["y"]).T
df.head()

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.3
1,0.004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.3
2,0.008,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.3
3,0.012,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.3
4,0.016,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.3


In [ ]:
dt = datetime.datetime.today()
seconds = dt.timestamp()
seconds + df[0]
df[0] = pd.to_datetime(df[0])


### Trial timings

- [0 , 2] (s) standard resting period
- [2 , 8-1/256] (s) show fixation cross
- [3 , 4.25] (s) show cue
- [8 , 9.5-10.5] (s) random intertrial interval

In [10]:
import warnings

# sample frequency
freq = 250 # Hz

# for all trials
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    for trial in dataframe_list:
        # create binary columns for each stage
        trial['Interval'] = [False]*trial.shape[0]
        trial['Cross'] = [False]*trial.shape[0]
        trial['Cue'] = [False]*trial.shape[0]

        # get the starting time of this particular trial
        t0 = trial['Time'].values[0] # 

        # calculate timings of stages (simple addition)
        start_interval, end_interval = t0, t0 + 2
        start_cross   , end_cross    = t0 + 2, t0 + 8-1/256
        start_cue     , end_cue      = t0 + 3, t0 + 4.25
        start_RIHI                   = t0 + 8

        # interval stage
        c1 = trial['Time'] >= start_interval
        c2 = trial['Time'] <= end_interval
        trial.loc[c1 & c2, 'Interval'] = True

        # cross stage
        c1 = trial['Time'] > start_cross
        c2 = trial['Time'] <= end_cross
        trial.loc[c1 & c2, 'Cross'] = True

        # cue stage
        c1 = trial['Time'] >= start_cue
        c2 = trial['Time'] <= end_cue
        trial.loc[c1 & c2, 'Cue'] = True

        # randomised interval after trial
        c1 = trial['Time'] >= start_RIHI
        trial.loc[c1, 'Interval'] = True

In [13]:
dataframe_list[0][['Time', 'Interval', 'Cross', 'Cue']].head()

,Time,Interval,Cross,Cue
3002,12.008,True,False,False
3003,12.012,True,False,False
3004,12.016,True,False,False
3005,12.020,True,False,False
3006,12.024,True,False,False
